# TREC 2018 Precision Medicine

In [1]:
from json2html import *
from IPython.display import HTML
import pandas
import os
import json
import xml.etree.ElementTree
import operator
import pytrec_eval
import eval
import utils

NameError: name 'config' is not defined

In [2]:
with open('config.json', 'r') as f:
    config = json.load(f)
HTML(json2html.convert(json = config))

ABSTRACTS,/abstracts
ELASTIC,http://trekkie.ddns.net:9200
TRIALS,/trials
FILENAME,YEAR.SUBTASK.RUNID.run
TOPICS_YEAR,2017


Load TOPICS and GOLD STANDARD from last year (30 topics)

In [3]:
topics_dict, topics_df = eval.get_topics('./topics/topics2017.xml')
qrels_dict = eval.get_qrels('./gold-standard/abstracts.2017.qrels')
assert(len(topics_dict)==len(qrels_dict))
topics_df

,topic,disease,gene,demographic
1,1,Liposarcoma,CDK4 Amplification,38-year-old male
2,2,Colon cancer,"KRAS (G13D), BRAF (V600E)",52-year-old male
3,3,Meningioma,"NF2 (K322), AKT1(E17K)",45-year-old female
4,4,Breast cancer,"FGFR1 Amplification, PTEN (Q171)",67-year-old female
5,5,Melanoma,"BRAF (V600E), CDKN2A Deletion",45-year-old female
6,6,Melanoma,NRAS (Q61K),55-year-old male
7,7,Lung cancer,EGFR (L858R),50-year-old female
8,8,Lung cancer,EML4-ALK Fusion transcript,52-year-old male
9,9,Gastrointestinal stromal tumor,KIT Exon 9 (A502_Y503dup),49-year-old female
10,10,Lung adenocarcinoma,KRAS (G12C),61-year-old female


In [4]:
run_dict_pytrec_eval, run_output_file = eval.run('my_run', topics_dict)

RUN: my_run
TOPIC: 1 Liposarcoma CDK4 Amplification 38-year-old male


NameError: name 'URL' is not defined

In [6]:
for topic_tuple in sorted(topics.items()):
    topic_num = topic_tuple[0]
    topic = topic_tuple[1]
    print("TOPIC:", str(topic_num))
    #print(topic['topic'], topic['disease'], topic['gene'], topic['demographic'])

    # For query...
    # Fill template with query
    with open('query.json', 'r') as myfile:
      query = myfile.read()  #print(data)
      query = query.replace('{{disease}}', topic['disease'])
      query = query.replace('{{gene}}', topic['gene'])
      query = query.replace('{{demographic}}', topic['demographic'])
      #print(query)

    response = requests.post(URL, data=query, headers=HEADERS)
    #print(response.json())

    run_id = "my_run"
    rank = 1

    for hit in response.json()["hits"]["hits"]:
        #print(topic_num, "Q0", hit["_id"], rank, hit["_score"], run)

        #Build list to export to file
        row_tuple = str(topic_num), "Q0", hit["_id"], str(rank), str(hit["_score"]), run_id
        run_output_file.append(row_tuple)
        rank = rank + 1

        #Build structure for python trec_eval
        if str(topic_num) not in run.keys():
            run[str(topic_num)] = {}
        run[str(topic_num)][hit["_id"]] = hit["_score"]

TOPIC: 1
TOPIC: 2
TOPIC: 3
TOPIC: 4
TOPIC: 5
TOPIC: 6
TOPIC: 7
TOPIC: 8
TOPIC: 9
TOPIC: 10
TOPIC: 11
TOPIC: 12
TOPIC: 13
TOPIC: 14
TOPIC: 15
TOPIC: 16
TOPIC: 17
TOPIC: 18
TOPIC: 19
TOPIC: 20
TOPIC: 21
TOPIC: 22
TOPIC: 23
TOPIC: 24
TOPIC: 25
TOPIC: 26
TOPIC: 27
TOPIC: 28
TOPIC: 29
TOPIC: 30


In [7]:
MEASURES_AGGREGATED = {'ndcg':'', 'Rprec':'', 'P_10':''}

evaluator = pytrec_eval.RelevanceEvaluator(qrel, pytrec_eval.supported_measures)
results = evaluator.evaluate(run)

for measure in MEASURES_AGGREGATED.keys():
    measure_all = pytrec_eval.compute_aggregated_measure(measure, [MEASURES_AGGREGATED[measure] for MEASURES_AGGREGATED in results.values()])
    MEASURES_AGGREGATED[measure] = round(measure_all, 4)
    #print(measure, 'all', measure_all)

for row in run_output_file:
    print(" ".join(row))

1 Q0 25028469 1 49.26335 my_run
1 Q0 11505267 2 45.88889 my_run
1 Q0 18551309 3 44.33857 my_run
1 Q0 26993784 4 44.038628 my_run
1 Q0 9713346 5 43.964874 my_run
1 Q0 15991843 6 43.38261 my_run
1 Q0 21317707 7 42.905556 my_run
1 Q0 26464734 8 42.880287 my_run
1 Q0 16489442 9 42.743355 my_run
1 Q0 18820664 10 42.538643 my_run
1 Q0 22528826 11 42.419 my_run
1 Q0 25594652 12 42.243446 my_run
1 Q0 16160477 13 41.670666 my_run
1 Q0 19823827 14 41.51307 my_run
1 Q0 9403703 15 41.052376 my_run
1 Q0 26336885 16 41.051132 my_run
1 Q0 20473880 17 41.020878 my_run
1 Q0 19574885 18 40.95031 my_run
1 Q0 18204431 19 40.933876 my_run
1 Q0 26645460 20 40.774376 my_run
1 Q0 24788530 21 40.039104 my_run
1 Q0 25765001 22 39.58143 my_run
1 Q0 21038090 23 39.543793 my_run
1 Q0 23826426 24 39.38463 my_run
1 Q0 21910158 25 39.38002 my_run
1 Q0 24457460 26 39.297344 my_run
1 Q0 15154619 27 39.19333 my_run
1 Q0 25986682 28 38.78408 my_run
1 Q0 23529275 29 38.427547 my_run
1 Q0 25993159 30 38.354504 my_run
1 Q0 

In [8]:
print(MEASURES_AGGREGATED)

{'P_10': 0.4767, 'Rprec': 0.2683, 'ndcg': 0.4994}
